In [5]:
import pandas as pd
import numpy as np
import requests
import random
import urllib
import json
import time
import sys
import datetime
from datetime import date
from selenium import webdriver
import os

class HelperClassTempo:
    """
    Classe de métodos auxiliares
    """
    def __init__(self):
        
        # mapeamento de meses
        self.dict_map_mes = {1: 'janeiro',
                             2: 'fevereiro',
                             3: 'março',
                             4: 'abril',
                             5: 'maio',
                             6: 'junho',
                             7: 'julho',
                             8: 'agosto',
                             9: 'setembro',
                             10: 'outubro',
                             11: 'novembro',
                             12: 'dezembro'
                             }
        
        # dia atual
        print (self.get_dia_atual())
        
        # path atual
        self.current_path = str(os.getcwd())
        
        # path do chromedriver
        self.path_to_chromedriver = os.path.join(self.current_path, 'chromedriver')
        
        # arquivos auxiliares
        self.path_infos_cidades = os.path.join(self.current_path, "cidades.csv")
        path_credenciais_user_agent = os.path.join(self.current_path, "credenciais_user_agent.json")
        
        # leitura do arquivo json com as credenciais
        try:
            f = open(path_credenciais_user_agent, mode="r")
            infos_login = json.load(f)
            self.user_agent = infos_login['user_agent']
            f.close()
        except:
            self.user_agent = "temporary_agent"
        
        # parametros do webdriver
        self.chromeOptions = webdriver.ChromeOptions()
        self.chromeOptions.add_argument('--no-sandbox')
        self.chromeOptions.add_argument("--headless")
        self.chromeOptions.add_argument(f"user-agent={self.user_agent}")
        
        # parâmetros
        self.url_tabua_mares = "https://www.tideschart.com"
        self.tempo_espera_tweet_segundos = 60
        self.qtd_cidades_selecionadas = 15
        self.qtd_min_dias_consecutivos = 10
        self.multiplicador_std = 1.3
        self.altura_mare_ruim = 1.6
        self.filler_tempo = 'céu nublado'
        self.modulo = 'tempo'
        
        # temperaturas max e min possíveis (validação conceitual)
        self.maior_temperatura_possivel = 40
        self.menor_temperatura_possivel = -10
        
        # icones
        self.icone_up = '▲'
        self.icone_down = '▼'
        
        # df cidades
        self.df_cidades = pd.read_csv(self.path_infos_cidades, encoding='latin-1', sep=';')
        
        # colunas para atribuir valor
        self.lista_colunas_tempo = ['cidade',
                                    'uf',
                                    'tempo',
                                    'temperatura',
                                    'temperatura_max',
                                    'temperatura_min',
                                    'nebulosidade',
                                    'umidade',
                                    'vento',
                                    'horario_por_sol',
                                    'pesca',
                                    'melhor_horario_pesca',
                                    'altura_maior_onda',
                                    'texto_onda',
                                    'url_imagem']
        
        # colunas para atribuir valor
        self.lista_colunas_salvar = ['cidade',
                                    'uf',
                                    'tempo',
                                    'temperatura',
                                    'temperatura_max',
                                    'temperatura_min',
                                    'nebulosidade',
                                    'umidade',
                                    'vento',
                                    'horario_por_sol',
                                    'pesca',
                                    'melhor_horario_pesca',
                                    'altura_maior_onda',
                                    'data']
        
        # colunas de clima
        f = open("mapeamento_climas.json", mode="r", encoding="utf-8")
        self.dict_map_clima = json.load(f)
        f.close()
        
        # colunas de pesca
        self.dict_map_pesca = {'Today is an excellent fishing day': 'Excelente',
                               'Today is a good fishing day': 'Bom',
                               'Today is an average fishing day': 'Mediano'}
        
        # path dos conteúdos do site
        self.path_url_imagem = '//*[@id="map"]'
        self.path_tempo1 = '//*[@id="main-content"]/div/div/div/div[1]/div[7]/div[1]/div/div[1]'
        self.path_temperatura1 = '//*[@id="main-content"]/div/div/div/div[1]/div[7]/div[2]/div/div[1]'
        self.path_temperatura_max_min1 = '//*[@id="main-content"]/div/div/div/div[1]/div[7]/div[2]/div/div[2]'
        self.path_nebulosidade1 = '//*[@id="main-content"]/div/div/div/div[1]/div[7]/div[1]/div/div[2]'
        self.path_umidade1 = '//*[@id="main-content"]/div/div/div/div[1]/div[7]/div[4]/div/div[1]'
        self.path_vento1 = '//*[@id="main-content"]/div/div/div/div[1]/div[7]/div[3]/div/div[1]'
        
        self.path_tempo2 = '//*[@id="main-content"]/div/div/div/div[1]/div[4]/div[1]/div/div[1]'
        self.path_temperatura2 = '//*[@id="main-content"]/div/div/div/div[1]/div[4]/div[2]/div/div[1]'
        self.path_temperatura_max_min2 = '//*[@id="main-content"]/div/div/div/div[1]/div[4]/div[2]/div/div[2]'
        self.path_nebulosidade2 = '//*[@id="main-content"]/div/div/div/div[1]/div[4]/div[1]/div/div[2]'
        self.path_umidade2 = '//*[@id="main-content"]/div/div/div/div[1]/div[4]/div[4]/div/div[1]'
        self.path_vento2 = '//*[@id="main-content"]/div/div/div/div[1]/div[4]/div[3]/div/div[1]'
        self.path_situacao_pesca1 = '//*[@id="main-content"]/div/div/div/div[1]/div[5]/div/h2/span'
        self.path_situacao_pesca2 = '//*[@id="main-content"]/div/div/div/div[1]/div[3]/div/h2/span'
        
        self.path_horario_por_sol = '//*[@id="main-content"]/div/div/div/div[1]/div[3]/div/div/table/tbody/tr[1]/td[7]'
        self.path_melhor_horario_pesca1 = '//*[@id="main-content"]/div/div/div/div[1]/div[5]/div/div/div[1]/div/h4'
        self.path_melhor_horario_pesca2 = '//*[@id="main-content"]/div/div/div/div[1]/div[5]/div/div/div[1]/div/h4[2]'
        self.path_por_sol = '//*[@id="main-content"]/div/div/div/div[1]/div[3]/div/div/table/tbody/tr[1]/td[7]'
        self.path_mare1 = '//*[@id="main-content"]/div/div/div/div[1]/div[3]/div/div/table/tbody/tr[1]/td[2]'
        self.path_mare2 = '//*[@id="main-content"]/div/div/div/div[1]/div[3]/div/div/table/tbody/tr[2]/td[3]'
        self.path_mare3 = '//*[@id="main-content"]/div/div/div/div[1]/div[3]/div/div/table/tbody/tr[1]/td[4]'
        self.path_mare4 = '//*[@id="main-content"]/div/div/div/div[1]/div[3]/div/div/table/tbody/tr[1]/td[5]'

    
    def get_dia_atual(self):
        '''
        data de hoje
        '''
        # data de hoje
        dia = date.today().strftime("%d")
        mes = self.dict_map_mes[int(date.today().strftime("%m"))]
        ano = date.today().strftime("%Y")
        return f"{dia} de {mes} de {ano}"
    
    
    def trata_mare(self, elemento):
        '''
        trata maré
        '''
        horario = (elemento.split('m')[0] + 'm').strip()
        altura = float(elemento.split(' ')[1].strip())
        simbolo = elemento.split(' ')[0][-1].strip()
        
        if simbolo == self.icone_up:
            simbolo = 'up'
        else:
            simbolo = 'down'
        return (horario, altura, simbolo)

    
    def gera_df_tabua_mares(self):
        '''
        Gera resultados dos climas
        '''
    
        lista_infos = []

        # itera cidades
        for index, row in self.df_cidades.iterrows():

            try:

                cidade = row['Cidade']
                uf = row['UF']
                valor = row['Tabua_mares']

                # cria urls
                url_dia = f"{self.url_tabua_mares}/{valor}"

                # entra na url
                driver = webdriver.Chrome(self.path_to_chromedriver, options=self.chromeOptions)
                driver.get(url_dia)
                time.sleep(2)

                # leitura do conteúdo
                try:
                    tempo = driver.find_element_by_xpath(self.path_tempo1).text
                except:
                    try:
                        tempo = driver.find_element_by_xpath(self.path_tempo2).text
                    except Exception as e:
                        print (f'Erro na cidade {cidade}! {e}')
                        continue
                    
                try:
                    temperatura = int((driver.find_element_by_xpath(self.path_temperatura1).text).split('°C')[0])
                except:
                    try:
                        temperatura =  int((driver.find_element_by_xpath(self.path_temperatura2).text).split('°C')[0])
                    except Exception as e:
                        print (f'Erro na cidade {cidade}! {e}')
                        continue
                    
                          
                try:
                    temperatura_max_min = driver.find_element_by_xpath(self.path_temperatura_max_min1).text
                except:
                    try:
                        temperatura_max_min = driver.find_element_by_xpath(self.path_temperatura_max_min2).text
                    except Exception as e:
                        print (f'Erro na cidade {cidade}! {e}')
                        continue
                    
                try:
                    nebulosidade = int(driver.find_element_by_xpath(self.path_nebulosidade1).text.split('Cloud cover ')[1].split('%')[0])
                except:
                    try:
                        nebulosidade = int(driver.find_element_by_xpath(self.path_nebulosidade2).text.split('Cloud cover ')[1].split('%')[0])
                    except Exception as e:
                        print (f'Erro na cidade {cidade}! {e}')
                        continue
                    
                
                try:
                    umidade = int(driver.find_element_by_xpath(self.path_umidade1).text.split('%')[0])
                except:
                    try:
                        umidade = int(driver.find_element_by_xpath(self.path_umidade2).text.split('%')[0])
                    except Exception as e:
                        print (f'Erro na cidade {cidade}! {e}')
                        continue
                     
                    
                try:
                    vento = int(driver.find_element_by_xpath(self.path_vento1).text.split(' ')[0])
                except:
                    try:
                        vento = int(driver.find_element_by_xpath(self.path_vento2).text.split(' ')[0])
                    except Exception as e:
                        print (f'Erro na cidade {cidade}! {e}')
                        continue
                    
                try:
                     pesca = driver.find_element_by_xpath(self.path_situacao_pesca1).text
                except:
                    try:
                        pesca = driver.find_element_by_xpath(self.path_situacao_pesca2).text
                    except Exception as e:
                        print (f'Erro na cidade {cidade}! {e}')
                        continue
                    
    
                # melhor horário de pesca
                try:
                    melhor_horario_pesca = driver.find_element_by_xpath(self.path_melhor_horario_pesca2).text.split('From ')[1].replace(' to ', ' - ')
                except:
                    try:
                        melhor_horario_pesca = driver.find_element_by_xpath(self.path_melhor_horario_pesca1).text.split('From ')[1].replace(' to ', ' - ')
                    except Exception as e:
                        melhor_horario_pesca = ''
                
                # trata melhor horário para pescar
                if melhor_horario_pesca != '':
                    try:
                        horario1, horario2 = melhor_horario_pesca.split("-")
                        horario1 = horario1.strip()
                        horario2 = horario2.strip()

                        # horario1
                        if "am" in horario1:
                            horario1 = horario1.split("am")[0]
                        elif "pm" in horario1:
                            horario1 = horario1.split("pm")[0]
                            hora, minuto = horario1.split(":")
                            hora = str(int(hora) + 12)
                            if int(hora) == 12:
                                hora = "0"
                            horario1 = f"{hora}:{minuto}"

                        # horario2
                        if "am" in horario2:
                            horario2 = horario2.split("am")[0]
                        elif "pm" in horario2:
                            horario2 = horario2.split("pm")[0]
                            hora, minuto = horario2.split(":")
                            hora = str(int(hora) + 12)
                            if int(hora) == 12:
                                hora = "0"
                            horario2 = f"{hora}:{minuto}"

                        melhor_horario_pesca = f"{horario1} - {horario2}"

                    except:
                        pass
                        
                # por do sol
                horario_por_sol = str(driver.find_element_by_xpath(self.path_horario_por_sol).text).split(' ')[1].strip()
                if "am" in horario_por_sol:
                    horario_por_sol = horario_por_sol.split("am")[0]
                elif "pm" in horario_por_sol:
                    horario_por_sol = horario_por_sol.split("pm")[0]
                    hora, minuto = horario_por_sol.split(":")
                    hora = str(int(hora) + 12)
                    horario_por_sol = f"{hora}:{minuto}"
                
                # marés
                try:
                    horario_mare_1, altura_mare_1, simbolo_mare_1 = self.trata_mare(driver.find_element_by_xpath(self.path_mare1).text)
                except:
                    horario_mare_1 = ""
                    altura_mare_1 = -1
                    simbolo_mare_1 = ""
                    
                try:
                    horario_mare_2, altura_mare_2, simbolo_mare_2 = self.trata_mare(driver.find_element_by_xpath(self.path_mare2).text)
                except:
                    horario_mare_2 = ""
                    altura_mare_2 = -1
                    simbolo_mare_2 = ""
                    
                try:
                    horario_mare_3, altura_mare_3, simbolo_mare_3 = self.trata_mare(driver.find_element_by_xpath(self.path_mare3).text)
                except:
                    horario_mare_3 = ""
                    altura_mare_3 = -1
                    simbolo_mare_3 = ""
                
                try:
                    horario_mare_4, altura_mare_4, simbolo_mare_4 = self.trata_mare(driver.find_element_by_xpath(self.path_mare4).text)
                
                except:
                    horario_mare_4 = ""
                    altura_mare_4 = -1
                    simbolo_mare_4 = ""
                    
                # altura da maior onda
                altura_maior_onda = str(max(altura_mare_1, altura_mare_2, altura_mare_3, altura_mare_4))

                # tratamento dos campos
                altura_maior_onda = str(altura_maior_onda).replace('.',',')
                altura_mare_1 = str(altura_mare_1).replace('.',',')
                altura_mare_2 = str(altura_mare_2).replace('.',',')
                altura_mare_3 = str(altura_mare_3).replace('.',',')
                altura_mare_4 = str(altura_mare_4).replace('.',',')
                
                # texto onda
                if (simbolo_mare_1 == ""):
                    continue
                
                elif (simbolo_mare_1 == "up" and simbolo_mare_2 == ""):
                     texto_onda = f"Horário de maré alta: {horario_mare_1} ({altura_mare_1}m)"
                        
                elif (simbolo_mare_1 == "down" and simbolo_mare_2 == ""):
                     texto_onda = f"Horário de maré baixa: {horario_mare_1} ({altura_mare_1}m)"
                              
                elif (simbolo_mare_1 == 'up' and simbolo_mare_4 != ""):
                    
                    texto_onda = f"Horários de maré alta: {horario_mare_1} ({altura_mare_1}m) e {horario_mare_3} ({altura_mare_3}m).\nHorários de maré baixa: {horario_mare_2} ({altura_mare_2}m) e {horario_mare_4} ({altura_mare_4}m)"
                
                elif (simbolo_mare_1 == 'up' and simbolo_mare_4 == ""):
                    texto_onda = f"Horários de maré alta: {horario_mare_1} ({altura_mare_1}m) e {horario_mare_3} ({altura_mare_3}m).\nHorário de maré baixa: {horario_mare_2} ({altura_mare_2}m)"
                    
                elif (simbolo_mare_1 == 'down' and simbolo_mare_4 != ""):
                    
                    texto_onda = f"Horário de maré alta: {horario_mare_2} ({altura_mare_2}m) e {horario_mare_4} ({altura_mare_4}m).\nHorários de maré baixa: {horario_mare_1} ({altura_mare_1}m) e {horario_mare_3} ({altura_mare_3}m)"
                
                elif (simbolo_mare_1 == 'down' and simbolo_mare_4 == ""):
                    texto_onda = f"Horário de maré alta: {horario_mare_2} ({altura_mare_2}m).\nHorários de maré baixa: {horario_mare_1} ({altura_mare_1}m) e {horario_mare_3} ({altura_mare_3}m)"
                    
                else:
                    continue
                  
                # temperatura max
                temperatura_max = int(temperatura_max_min.split('Min ')[1].split('°')[0])
                
                # temperatura min
                temperatura_min = int(temperatura_max_min.split('Max ')[1].split('°')[0])
                
                # validação conceitual de temperaturas
                maior_valor_temperatura = max([temperatura, temperatura_max, temperatura_min])
                menor_valor_temperatura = min([temperatura, temperatura_max, temperatura_min])                         

                # se estiver fora da range, ignora a cidade e continua o processo
                if (maior_valor_temperatura > self.maior_temperatura_possivel or menor_valor_temperatura < self.menor_temperatura_possivel):
                    continue
                    
                # mapeia tempo
                if tempo in self.dict_map_clima.keys():
                    try:
                        tempo = self.dict_map_clima[tempo]
                    except:
                        tempo = self.filler_tempo
                else:
                    tempo = self.filler_tempo
                     
                # mapeia pesca
                if pesca in self.dict_map_pesca.keys():
                    try:
                        pesca = self.dict_map_pesca[pesca]
                    except:
                        pesca = ''
                else:
                    pesca = ''
                      
                    
                # imagem adicional do litoral da cidade
                url_imagem = driver.find_element_by_xpath(self.path_url_imagem).get_attribute("src")
                
                # salva lista
                lista_infos.append([cidade,
                                    uf,
                                    tempo,
                                    temperatura,
                                    temperatura_max,
                                    temperatura_min,
                                    nebulosidade,
                                    umidade,
                                    vento,
                                    horario_por_sol,
                                    pesca,
                                    melhor_horario_pesca,
                                    altura_maior_onda,
                                    texto_onda,
                                    url_imagem])

            # erro de execução
            except Exception as e:
                print (f'Erro na cidade {cidade}! {e}')
                continue

        # fecha o driver
        driver.close()
                                                  
        # cria o dataframe
        try:
            df_infos = pd.DataFrame(lista_infos,
                                    columns=self.lista_colunas_tempo)

            # tratamentos adicionais
            df_infos['temperatura_max'] = df_infos[["temperatura", "temperatura_max", "temperatura_min"]].max(axis=1)
            df_infos['temperatura_min'] = df_infos[["temperatura", "temperatura_max", "temperatura_min"]].min(axis=1)

            # transforma tudo em string
            for column in df_infos.columns.tolist():
                df_infos[column] = df_infos[column].astype(str)
                
            # inclui data de hoje
            df_infos['data'] = date.today().strftime("%d/%m/%Y")

            # retorna resultados
            return df_infos
        
        # erro de execução
        except Exception as e:
            print (f'Erro no fim do processo! {e}')
            sys.exit(0)

# gera resultados
objeto = HelperClassTempo()
df_resultados = objeto.gera_df_tabua_mares()
df_resultados

24 de abril de 2022


/tmp/ipykernel_127609/2309673606.py:202: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(self.path_to_chromedriver, options=self.chromeOptions)
/tmp/ipykernel_127609/2309673606.py:208: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  tempo = driver.find_element_by_xpath(self.path_tempo1).text
/tmp/ipykernel_127609/2309673606.py:217: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  temperatura = int((driver.find_element_by_xpath(self.path_temperatura1).text).split('°C')[0])
/tmp/ipykernel_127609/2309673606.py:227: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  temperatura_max_min = driver.find_element_by_xpath(self.path_temperatura_max_min1).text
/tmp/ipykernel_127609/2309673606.py:236: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_eleme

,cidade,uf,tempo,temperatura,temperatura_max,temperatura_min,nebulosidade,umidade,vento,horario_por_sol,pesca,melhor_horario_pesca,altura_maior_onda,texto_onda,url_imagem,data
0,Angra dos Reis,RJ,chuva fraca,23,24,23,100,85,21,17:37,Mediano,18:37 - 20:37,"0,9","Horários de maré alta: 1:08am (0,9m) e 8:57am ...",https://www.tideschart.com/maps/en/Angra-Dos-R...,24/04/2022
1,Aracaju,SE,céu parcialmente nublado [emoji_nuvem],28,28,28,17,66,22,17:20,Mediano,18:06 - 20:06,"0,7","Horário de maré alta: 6:47am (0,6m).\nHorários...",https://www.tideschart.com/maps/en/Aracaju-Ser...,24/04/2022
2,Arraial do Cabo,RJ,céu parcialmente nublado [emoji_nuvem],24,26,24,49,80,26,17:28,Mediano,18:28 - 20:28,"0,9","Horários de maré alta: 12:36am (0,9m) e 8:25am...",https://www.tideschart.com/maps/en/Arraial-Do-...,24/04/2022
3,Balneário Camboriu,SC,céu parcialmente nublado [emoji_nuvem],21,21,21,70,85,16,17:49,Mediano,18:55 - 20:55,"0,8","Horários de maré alta: 12:41am (0,8m) e 8:30am...",https://www.tideschart.com/maps/en/Balneario-C...,24/04/2022
4,Barreirinhas,MA,períodos de chuva leve ao longo do dia [emoji_...,25,27,25,58,85,19,17:48,Mediano,18:27 - 20:27,"0,9","Horário de maré alta: 7:06am (0,8m).\nHorários...",https://www.tideschart.com/maps/en/Atins-Barre...,24/04/2022
5,Bertioga,SP,períodos de chuva ao longo do dia [emoji_chuva],23,24,23,81,86,24,17:43,Mediano,18:45 - 20:45,"1,2","Horário de maré alta: 10:56am (1,1m) e 11:21pm...",https://www.tideschart.com/maps/en/Bertioga-Sa...,24/04/2022
6,Bombinhas,SC,céu parcialmente nublado [emoji_nuvem],20,22,20,73,87,18,17:49,Mediano,18:55 - 20:55,"0,8","Horários de maré alta: 1:11am (0,8m) e 9:00am ...",https://www.tideschart.com/maps/en/Bombinhas-S...,24/04/2022
7,Búzios,RJ,céu parcialmente nublado [emoji_nuvem],24,25,24,51,82,26,17:27,Mediano,18:27 - 20:27,"1,0","Horários de maré alta: 12:34am (1,0m) e 8:23am...",https://www.tideschart.com/maps/en/Armacao-De-...,24/04/2022
8,Cabo Frio,RJ,céu parcialmente nublado [emoji_nuvem],24,26,24,49,80,26,17:28,Mediano,18:28 - 20:28,"0,9","Horários de maré alta: 12:36am (0,9m) e 8:25am...",https://www.tideschart.com/maps/en/Cabo-Frio-A...,24/04/2022
9,Enseada,SP,chuva fraca,22,24,22,100,90,27,17:39,Mediano,18:40 - 20:40,"1,0","Horários de maré alta: 1:01am (1,0m) e 8:50am ...",https://www.tideschart.com/maps/en/Praia-da-En...,24/04/2022
